# Homework - 3


https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv was downloaded as the data.csv file in this notebook.

In [1]:
import pandas as pd

data=pd.read_csv("data.csv")
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [2]:


data.columns = data.columns.str.lower().str.replace(' ', '_')
data['year']=data['year'].astype(str) 
data=data[['make','model','year','engine_hp','engine_cylinders','transmission_type','vehicle_style','highway_mpg','city_mpg','msrp']]
categorical_columns = list(data.dtypes[data.dtypes == 'object'].index)


for c in categorical_columns:
    data[c] = data[c].str.lower().str.replace(' ', '_')
   
data.isna().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   make               11914 non-null  object 
 1   model              11914 non-null  object 
 2   year               11914 non-null  object 
 3   engine_hp          11845 non-null  float64
 4   engine_cylinders   11884 non-null  float64
 5   transmission_type  11914 non-null  object 
 6   vehicle_style      11914 non-null  object 
 7   highway_mpg        11914 non-null  int64  
 8   city_mpg           11914 non-null  int64  
 9   msrp               11914 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 930.9+ KB


In [4]:
data.fillna(0,inplace=True)
data.isna().sum()


make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
msrp                 0
dtype: int64

In [5]:
data.rename(columns={'msrp': 'price'}, inplace=True)
data.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,19,46135
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,19,40650
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,20,36350
3,bmw,1_series,2011,230.0,6.0,manual,coupe,28,18,29450
4,bmw,1_series,2011,230.0,6.0,manual,convertible,28,18,34500


# Question 1
What is the most frequent observation (mode) for the column transmission_type?

In [6]:
data['transmission_type'].mode()

0    automatic
Name: transmission_type, dtype: object

# Question 2
Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

In [7]:
correlation_engine_hp_year = data[['engine_hp', 'year']].corr().values[0,1]
correlation_engine_hp_engine_cylinders = data[['engine_hp', 'engine_cylinders']].corr().values[0,1]
correlation_highway_mpg_engine_cylinders = data[['highway_mpg', 'engine_cylinders']].corr().values[0,1]
correlation_highway_mpg_city_mpg = data[['highway_mpg', 'city_mpg']].corr().values[0,1]


In [8]:
correlation_engine_hp_year,correlation_engine_hp_engine_cylinders,correlation_highway_mpg_engine_cylinders,correlation_highway_mpg_city_mpg

(0.33871418476243353,
 0.7748509807813221,
 -0.6145414173953352,
 0.8868294962591425)

Highest is correlation with highway_mpg and city_mpg

## Make price binary
Now we need to turn the price variable from numeric into a binary format.

Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.

## Split the data
Split your data in train/val/test sets with 60%/20%/20% distribution.

Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.

Make sure that the target value (price) is not in your dataframe.

In [9]:
mean_price = data['price'].mean()
from sklearn.model_selection import train_test_split


features = data.drop(columns=['price'], axis=1).copy()
target = (data['price'] > mean_price).astype(int)


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Question 3
- Calculate the mutual information score between above_average and other categorical variables in our dataset. 
- Use the training set only.Round the scores to 2 decimals using round(score, 2).

Which of these variables has the lowest mutual information score?

- make
- model
- transmission_type
- vehicle_style

In [10]:
from sklearn.metrics import mutual_info_score

def mutual_info_price_score(series):
    return round(mutual_info_score(series, y_train),2)

mutual_info_price_score(X_train.make),mutual_info_price_score(X_train.model),mutual_info_price_score(X_train.transmission_type),mutual_info_price_score(X_train.vehicle_style)


(0.24, 0.46, 0.02, 0.08)

The lowest is in transmission type

# Question 4
- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
- To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
   - model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [26]:
X_encoded = pd.get_dummies(features, columns=categorical_columns, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
original_accuracy = round(accuracy_score(y_val, y_pred),4)
print(f"Accuracy on the validation dataset: {original_accuracy}")

Accuracy on the validation dataset: 0.9446


# Question 5
 - Let's find the least useful feature using the feature elimination technique.
 - Train a model with all these features (using the same parameters as in Q4).
 - Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
 - For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

In [17]:
features

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,19
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,19
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,20
3,bmw,1_series,2011,230.0,6.0,manual,coupe,28,18
4,bmw,1_series,2011,230.0,6.0,manual,convertible,28,18
...,...,...,...,...,...,...,...,...,...
11909,acura,zdx,2012,300.0,6.0,automatic,4dr_hatchback,23,16
11910,acura,zdx,2012,300.0,6.0,automatic,4dr_hatchback,23,16
11911,acura,zdx,2012,300.0,6.0,automatic,4dr_hatchback,23,16
11912,acura,zdx,2013,300.0,6.0,automatic,4dr_hatchback,23,16


In [18]:
loop_features=features.copy()

In [27]:
import numpy as np
err={}
for i in loop_c:
    # print("excluding :",i)
    newf=features.drop(columns=i,axis=1)
    categorical_columns=list(newf.dtypes[newf.dtypes=='object'].index)
    X_encoded = pd.get_dummies(newf, columns=categorical_columns, drop_first=True)
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, target, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy =np.abs(original_accuracy- round(accuracy_score(y_val, y_pred), 4))
    err[i]=accuracy
    # print(f"Accuracy on the validation dataset : {accuracy}")
    # print("*"*15)



sorted_dict = dict(sorted(err.items(), key=lambda x: x[1]))
sorted_dict

{'vehicle_style': 0.00039999999999995595,
 'city_mpg': 0.00039999999999995595,
 'highway_mpg': 0.0008000000000000229,
 'transmission_type': 0.0017000000000000348,
 'engine_cylinders': 0.0020999999999999908,
 'make': 0.0029000000000000137,
 'year': 0.0029000000000000137,
 'engine_hp': 0.0046000000000000485,
 'model': 0.011299999999999977}

out of all the options, city_mpg has the lowest difference in the accuracy

# Question 6
- For this question, we'll see how to use a linear regression model from Scikit-Learn.
- We'll need to use the original column price. Apply the logarithmic transformation to this column.
- Fit the Ridge regression model on the training data with a solver 'sag'. Set the seed to 42.
- This model also has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10].
- Round your RMSE scores to 3 decimal digits.

In [14]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.metrics import mean_squared_error

# data['price'] = np.log1p(data['price'])
# X_en=pd.get_dummies(data, columns=categorical_columns, drop_first=True)
X = data.drop(columns=['price']+categorical_columns).copy()  # Features
y = np.log1p(data['price'])# Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
alpha_values = [0, 0.01, 0.1, 1, 10]
rmse_scores = {}

for alpha in alpha_values:
    model = Ridge(alpha=alpha, solver='sag', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores[alpha] = round(rmse, 3)

for alpha, rmse in rmse_scores.items():
    print(f"Alpha = {alpha}: RMSE = {rmse}")

Alpha = 0: RMSE = 0.759
Alpha = 0.01: RMSE = 0.759
Alpha = 0.1: RMSE = 0.759
Alpha = 1: RMSE = 0.759
Alpha = 10: RMSE = 0.759


In [16]:
from sklearn.linear_model import Ridge
import numpy as np
from sklearn.metrics import mean_squared_error


X_en=pd.get_dummies(data, columns=categorical_columns, drop_first=True)
X = X_en.drop(columns=['price']).copy()  # Features
y = np.log1p(data['price'])# Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
alpha_values = [0, 0.01, 0.1, 1, 10]
rmse_scores = {}

for alpha in alpha_values:
    model = Ridge(alpha=alpha, solver='sag', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores[alpha] = round(rmse, 3)

for alpha, rmse in rmse_scores.items():
    print(f"Alpha = {alpha}: RMSE = {rmse}")

c:\Users\harin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\harin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\harin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\harin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Alpha = 0: RMSE = 0.603
Alpha = 0.01: RMSE = 0.603
Alpha = 0.1: RMSE = 0.603
Alpha = 1: RMSE = 0.603
Alpha = 10: RMSE = 0.604


c:\Users\harin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


There is no significant improvement in the algorithm in varying the alpha values

Even when we include the categorical - one hot encoding approach and neglecting those categorical columns in the data frame of features.

thus we consider the lowest ridge value of 0.